In [44]:
#Starter code provided by Instructor
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import maps_api_key

In [45]:
path = "output_data/city_data.csv"
city_df = pd.read_csv(path)
city_df.head(5)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Victor Harbor,-35.57,138.62,55.99,74,96,4.00,AU,10/23/20
1,Pisco,-13.70,-76.22,71.60,60,0,16.11,PE,10/23/20
2,Port Keats,-14.25,129.55,81.34,79,11,8.61,AU,10/23/20
3,Qinhuangdao,39.93,119.59,45.30,53,0,5.19,CN,10/23/20
4,Margate,51.38,1.39,54.00,83,46,1.99,GB,10/23/20


In [46]:
# Configure gmaps
gmaps.configure(api_key=maps_api_key)

In [47]:
# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

In [48]:
# Plot Heatmap
fp = (0,0)
basic_layer = gmaps.symbol_layer(
    locations, scale=3)

fig = gmaps.figure(center=fp, zoom_level=2.0)
fig


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [49]:
#Creating ideal condidtions dataframe
ideal_conditions_df = city_df

ideal_conditions_df = ideal_conditions_df.loc[ideal_conditions_df['Max Temp'] > 75]
ideal_conditions_df = ideal_conditions_df.loc[ideal_conditions_df['Max Temp'] < 85]
ideal_conditions_df = ideal_conditions_df.loc[ideal_conditions_df['Humidity'] < 35]
ideal_conditions_df = ideal_conditions_df.loc[ideal_conditions_df['Cloudiness'] == 0]
ideal_conditions_df = ideal_conditions_df.loc[ideal_conditions_df['Wind Speed'] < 8.5]

In [50]:
ideal_conditions_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
161,Sekoma,-24.40,23.88,83.66,21,0,5.21,BW,10/23/20
162,Mount Darwin,-16.77,31.58,76.44,29,0,5.64,ZW,10/23/20
251,Morār,26.23,78.23,76.10,32,0,2.28,IN,10/23/20
258,Molina,-35.12,-71.28,78.80,31,0,6.93,CL,10/23/20
287,Buin,-33.73,-70.75,82.99,30,0,8.05,CL,10/23/20
378,San Bernardo,-33.60,-70.72,82.99,30,0,8.05,CL,10/23/20


In [51]:
#Creating the hotel df and adding a new column
hotel_df = ideal_conditions_df.loc[:, ['City', 'Country', 'Lat', 'Lng']]

In [52]:
hotel_df['Hotel Name'] = ''

In [53]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
161,Sekoma,BW,-24.40,23.88,
162,Mount Darwin,ZW,-16.77,31.58,
251,Morār,IN,26.23,78.23,
258,Molina,CL,-35.12,-71.28,
287,Buin,CL,-33.73,-70.75,
378,San Bernardo,CL,-33.60,-70.72,


In [54]:
#Using the nearby google search, loop through the city and find the nearest hotel
hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {'type' : 'hotel',
          'keyword' : 'hotel',
          'radius' : 5000,
          'key' : maps_api_key}

print('Beginning Data Retrieval')      
print('-----------------------------')

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    city_name = row['City']
    
    params['location'] = f'{lat},{lng}'
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(hotel_url, params=params).json() 
    
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print('Hotel not found. Skipping...')
        
print('-----------------------------')
print('Data Retrieval Complete')      
print('-----------------------------')   

Beginning Data Retrieval
-----------------------------
Retrieving Results for Index 161: Sekoma.
Hotel not found. Skipping...
Retrieving Results for Index 162: Mount Darwin.
Hotel not found. Skipping...
Retrieving Results for Index 251: Morār.
Closest hotel in Morār is Radisson Gwalior.
Retrieving Results for Index 258: Molina.
Closest hotel in Molina is Lagar Hotel Boutique..
Retrieving Results for Index 287: Buin.
Closest hotel in Buin is Paihuen.
Retrieving Results for Index 378: San Bernardo.
Hotel not found. Skipping...
-----------------------------
Data Retrieval Complete
-----------------------------


In [55]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
161,Sekoma,BW,-24.40,23.88,
162,Mount Darwin,ZW,-16.77,31.58,
251,Morār,IN,26.23,78.23,Radisson Gwalior
258,Molina,CL,-35.12,-71.28,Lagar Hotel Boutique.
287,Buin,CL,-33.73,-70.75,Paihuen
378,San Bernardo,CL,-33.60,-70.72,


In [56]:
#Droping cities that did not return a hotel
hotel_df = hotel_df.drop(hotel_df[hotel_df['Hotel Name'] == ''].index)
hotel_df

,City,Country,Lat,Lng,Hotel Name
251,Morār,IN,26.23,78.23,Radisson Gwalior
258,Molina,CL,-35.12,-71.28,Lagar Hotel Boutique.
287,Buin,CL,-33.73,-70.75,Paihuen


In [57]:
#Code provided by Instructor
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [58]:
# Add marker layer and info box
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

fig



Figure(layout=FigureLayout(height='420px'))